# Notes - Xarray

## Terminology

- DataArray: A multi-dimensional array with labeled or named dimensions. DataArray objects add metadata such as dimension names, coordinates, and attributes. For example, an array is var(time, level, lat, lon).
- DataSet: A dict-like collection of DataArray objects with aligned dimensions. For example, a dataset contains temperature(time, level, lat, lon) and precipitation(time, lat, lon).

## References

- Unidata Xarray Introduction, https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/
- Xarray quick overview, https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html
- Xarray computation, https://docs.xarray.dev/en/stable/user-guide/computation.html


In [1]:
import numpy as np
import xarray as xr
import io, os, sys, types
import yhc_module as yhc

## Create and Modify a DataArray

### Create a DataArray

Xarray - https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html#create-a-dataarray

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#DataArray

In [2]:
#--- Create some sample "temperature" data
data = 283 + 5 * np.random.randn(5, 3, 4)

time = np.arange(0,5)
lat = np.linspace(-120., 60., 3)
lon = np.linspace(25.,55.,4)

#--- create a DataArray & set attributes
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'], coords=[time, lat, lon])

temp.attrs['units'] = "K"
temp.attrs['long_name'] = "Temperature"

with xr.set_options(keep_attrs=True):  # keep attributes after operation
  temp_degC = temp - 273.15
temp_degC.attrs['units'] = "C"
yhc.printv(temp_degC, "temp_degC")

#--- put all in one line
arr = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
yhc.printv(arr, "arr")


--------------
temp_degC
<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 7.87850085,  1.43301462, 11.02999447,  5.05624882],
        [ 8.42889749,  8.33398687, 15.39192754, 25.28373801],
        [16.31759294, 12.82068889,  0.95594452,  8.7173809 ]],

       [[15.58084367,  5.39573536,  4.20243212,  8.40066904],
        [17.81799538, 10.01417624,  4.48276802, 12.4077183 ],
        [10.93780335, -2.88490077, 13.56841656, 10.31238807]],

       [[ 7.24349352,  8.99645204,  7.04388358, 15.65421046],
        [12.4099551 ,  5.21665837, 13.04596614, 10.35411415],
        [ 3.32270519,  7.10310441, 10.4391033 ,  5.32291649]],

       [[ 7.54218821,  8.48625556, 10.20926842, 11.43231246],
        [15.3338544 ,  4.15960464, 14.63617415,  4.06444006],
        [13.16225515, 10.78648837, 10.72504477, 20.98522227]],

       [[ 5.66993312,  8.2987517 , 10.1767648 , 17.34877253],
        [ 6.35683287, 17.28362479, 12.75283313, -2.36205411],
        [15.80956152,  5.87363888, 10.90505009,  8.791

### Modify coordinates of a DataArray

Xarray, modify coordinates
https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html

In [3]:
#--- create a sample
arr = xr.DataArray(np.random.RandomState(0).randn(2, 3), [("x", ["a", "b"]), ("y", [10, 20, 30])])
yhc.printv(arr, "original")

#--- print out coordinate
arr.coords["x"]   # print out coordinate "x" values

#--- assign completely new coordinate values
x_new = ["cc","dd"]
arr = arr.assign_coords({"x":x_new}) # modidy the arraymodify "x" coordinate from [a,b] to [c,d]
    #arr.assign_coords({"x":x_new})    # This will show new coordinate but not modidy the array
yhc.printv(arr,"modify x")

#--- arthemetric operation of exisitng coordiate values
arr.assign_coords(y=arr.y+1)   # modify the coordinate values
arr = arr.assign_coords(y=((arr.y+5)/2)+1.)
yhc.printv(arr,"modify y")

#--- put all in one line
arr = arr.assign_coords({"x":["ee","ff"], "y":arr.y+1})
yhc.printv(arr, "modify x and y")

#--- change coordinate names
arr = arr.rename({"x":"x2", "y":"y2"})  # rename coordinate
yhc.printv(arr, 'rename')

#--- New coordinate can also be attached to an existing dimension
y_new = [11,12,13]
arr = arr.assign_coords(y_new=("y2", y_new))
yhc.printv(arr, 'add y_new as a coordinate variable')

#--- Add a new coordinate and the original array would be expanded 
z = np.array([1,2,3,4])
arr = arr.expand_dims(z=z)   # add a new coordinate "z". The "z" values are np.arrry *z*.
yhc.printv(arr, "add a new z coordiate")


--------------
original
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30

--------------
modify x
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'cc' 'dd'
  * y        (y) int64 10 20 30

--------------
modify y
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'cc' 'dd'
  * y        (y) float64 8.5 13.5 18.5

--------------
modify x and y
<xarray.DataArray (x: 2, y: 3)>
array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])
Coordinates:
  * x        (x) <U2 'ee' 'ff'
  * y        (y) float64 9.5 14.5 19.5

--------------
rename
<xarray.DataArray (x2: 2, y2: 3)>
array(

## Selection

Unidata - https://unidata.github.io/python-training/workshop/XArray/xarray-introduction/#Selection

### Selection Method 1: use indexing

In [4]:
#--- Method 1: use indexing
var = temp[0, 1:2, :]
var

<xarray.DataArray (lat: 1, lon: 4)>
array([[281.57889749, 281.48398687, 288.54192754, 298.43373801]])
Coordinates:
    time     int64 0
  * lat      (lat) float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

### Selection Method 2: Use name dimension & slicing

In [5]:
#--- Method 2; use name dimension
temp.coords  # check out variable dimension
print(temp.coords)

#--- select specific values in coordinates
var = temp.sel(time=1, lat=-30., lon=25)
var = temp.sel(time=1, lat=-30.2, lon=25.3, method='nearest')

print('------------')
print(var)

var = temp.sel(time=1, lon=25)
print('------------')
print(var)

#--- Slicing with Selection
var = temp.sel(time=slice(0,2), lat=-30., lon=slice(-1000.,1000.))
print('------------')
print(var)

Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
------------
<xarray.DataArray ()>
array(290.96799538)
Coordinates:
    time     int64 1
    lat      float64 -30.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (lat: 3)>
array([288.73084367, 290.96799538, 284.08780335])
Coordinates:
    time     int64 1
  * lat      (lat) float64 -120.0 -30.0 60.0
    lon      float64 25.0
Attributes:
    units:      K
    long_name:  Temperature
------------
<xarray.DataArray (time: 3, lon: 4)>
array([[281.57889749, 281.48398687, 288.54192754, 298.43373801],
       [290.96799538, 283.16417624, 277.63276802, 285.5577183 ],
       [285.5599551 , 278.36665837, 286.19596614, 283.50411415]])
Coordinates:
  * time     (time) int64 0 1 2
    lat      float64 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_n

### Selection Method 3: use .loc

In [6]:
#*** Useful if already knowing the range to each coordinate

# temp is temp(time, lat, lon)
var = temp.loc[0:4, -120:30, :]
print(var)

<xarray.DataArray (time: 5, lat: 2, lon: 4)>
array([[[281.02850085, 274.58301462, 284.17999447, 278.20624882],
        [281.57889749, 281.48398687, 288.54192754, 298.43373801]],

       [[288.73084367, 278.54573536, 277.35243212, 281.55066904],
        [290.96799538, 283.16417624, 277.63276802, 285.5577183 ]],

       [[280.39349352, 282.14645204, 280.19388358, 288.80421046],
        [285.5599551 , 278.36665837, 286.19596614, 283.50411415]],

       [[280.69218821, 281.63625556, 283.35926842, 284.58231246],
        [288.4838544 , 277.30960464, 287.78617415, 277.21444006]],

       [[278.81993312, 281.4487517 , 283.3267648 , 290.49877253],
        [279.50683287, 290.43362479, 285.90283313, 270.78794589]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature


### Selection Method 4: where() to conditionally switch between values

In [7]:
#--- use where()
var = temp.where(temp > 280.)  # if temp < 280., it would become nan
var

#--- create a mask, but fail.
#xr.where(temp > 280., "positive", "negative")

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[281.02850085,          nan, 284.17999447,          nan],
        [281.57889749, 281.48398687, 288.54192754, 298.43373801],
        [289.46759294, 285.97068889,          nan, 281.8673809 ]],

       [[288.73084367,          nan,          nan, 281.55066904],
        [290.96799538, 283.16417624,          nan, 285.5577183 ],
        [284.08780335,          nan, 286.71841656, 283.46238807]],

       [[280.39349352, 282.14645204, 280.19388358, 288.80421046],
        [285.5599551 ,          nan, 286.19596614, 283.50411415],
        [         nan, 280.25310441, 283.5891033 ,          nan]],

       [[280.69218821, 281.63625556, 283.35926842, 284.58231246],
        [288.4838544 ,          nan, 287.78617415,          nan],
        [286.31225515, 283.93648837, 283.87504477, 294.13522227]],

       [[         nan, 281.4487517 , 283.3267648 , 290.49877253],
        [         nan, 290.43362479, 285.90283313,          nan],
        [288.95956152,          nan, 284.05505009, 281.94132532]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4
  * lat      (lat) float64 -120.0 -30.0 60.0
  * lon      (lon) float64 25.0 35.0 45.0 55.0
Attributes:
    units:      K
    long_name:  Temperature

## Reshaping and reorganizing data
https://docs.xarray.dev/en/stable/user-guide/reshaping.html#reordering-dimensions



## Computation &

### Basic numerical operation

In [8]:
#--- arithmetic operation
xr.set_options(keep_attrs =True)  # keep attributes after operations
temp_degC = temp - 273.15
temp_degC.attrs['units']="C"
temp_degC

#--- mean and others
#    a list: mean, min, max, std, sum, weighted
temp_tavg = temp.mean("time")
temp_tavg

temp_tiavg = temp.mean(["time","lon"])
temp_tiavg

<xarray.DataArray (lat: 3)>
array([282.00398627, 283.92066058, 282.84858651])
Coordinates:
  * lat      (lat) float64 -120.0 -30.0 60.0
Attributes:
    units:      K
    long_name:  Temperature

### Rolling average

https://xarray.pydata.org/en/stable/user-guide/computation.html#rolling-window-operations
https://xarray.pydata.org/en/stable/generated/xarray.DataArray.rolling.html#xarray.DataArray.rolling

In [9]:
  #--- pk values. Output directly from AM4 files
  pk_list = [100, 400, 818.6021, 1378.886, 2091.795, 2983.641, 4121.79, 5579.222, 
    6907.19, 7735.787, 8197.665, 8377.955, 8331.696, 8094.722, 7690.857, 
    7139.018, 6464.803, 5712.357, 4940.054, 4198.604, 3516.633, 2905.199, 
    2366.737, 1899.195, 1497.781, 1156.253, 867.792, 625.5933, 426.2132, 
    264.7661, 145.0665, 60, 15, 0]

  #--- bk values. Output directly from AM4 files
  bk_list = [0, 0, 0, 0, 0, 0, 0, 0, 0.00513, 0.01969, 0.04299, 0.07477, 0.11508, 
    0.16408, 0.22198, 0.28865, 0.36281, 0.44112, 0.51882, 0.59185, 0.6581, 
    0.71694, 0.76843, 0.81293, 0.851, 0.88331, 0.91055, 0.93331, 0.95214, 
    0.9675, 0.97968, 0.98908, 0.99575, 1]  

  #--- make DataArray
  ps = xr.DataArray([102000.], dims=['time'])
  pk = xr.DataArray(pk_list, dims=['plev'])
  bk = xr.DataArray(bk_list, dims=['plev'])

  #--- p at half levels: p = ps*bk + pk. ps is the first term because it would be broadcasted.
  phalf = ps*bk + pk

  #--- compute p at full levels, using .rolling(). NaNs is removed by dropna()
  pfull = phalf.rolling(plev=2, center=True).mean().dropna("plev")
  yhc.printv(phalf, "phalf")
  yhc.printv(pfull, "pfull")

--------------
phalf
<xarray.DataArray (time: 1, plev: 34)>
array([[1.00000000e+02, 4.00000000e+02, 8.18602100e+02, 1.37888600e+03,
        2.09179500e+03, 2.98364100e+03, 4.12179000e+03, 5.57922200e+03,
        7.43045000e+03, 9.74416700e+03, 1.25826450e+04, 1.60044950e+04,
        2.00698560e+04, 2.48308820e+04, 3.03328170e+04, 3.65813180e+04,
        4.34714230e+04, 5.07065970e+04, 5.78596940e+04, 6.45673040e+04,
        7.06428330e+04, 7.60330790e+04, 8.07465970e+04, 8.48180550e+04,
        8.82997810e+04, 9.12538730e+04, 9.37438920e+04, 9.58232133e+04,
        9.75444932e+04, 9.89497661e+04, 1.00072427e+05, 1.00946160e+05,
        1.01581500e+05, 1.02000000e+05]])
Dimensions without coordinates: time, plev

--------------
pfull
<xarray.DataArray (time: 1, plev: 33)>
array([[   250.     ,    609.30105,   1098.74405,   1735.3405 ,
          2537.718  ,   3552.7155 ,   4850.506  ,   6504.836  ,
          8587.3085 ,  11163.406  ,  14293.57   ,  18037.1755 ,
         22450.369  ,  275

### Broadcasting by dimension name

https://docs.xarray.dev/en/stable/user-guide/computation.html#broadcasting-by-dimension-name

In [10]:
a = xr.DataArray([1, 2], [("x", ["a", "b"])])
b = xr.DataArray([-1, -2, -3], [("y", [10, 20, 30])])
c = xr.DataArray(np.arange(6).reshape(3, 2), [b["y"], a["x"]])

#print(a)
#print(b)
#print(c)

#--- With xarray, and their dimensions are expanded automatically:
print(a*b)  # 2x3 array
print(b*a)  # 3x2 array

#--- explicitly broadcast xarray data structures by using the broadcast()
a2, b2 = xr.broadcast(a, b)
print(a2)
print(b2)


<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-2, -4, -6]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (y: 3, x: 2)>
array([[-1, -2],
       [-2, -4],
       [-3, -6]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'
<xarray.DataArray (x: 2, y: 3)>
array([[1, 1, 1],
       [2, 2, 2]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 10 20 30
<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-1, -2, -3]])
Coordinates:
  * y        (y) int64 10 20 30
  * x        (x) <U1 'a' 'b'


### Automatic alignment
https://docs.xarray.dev/en/stable/user-guide/computation.html#automatic-alignment



In [11]:
arr = xr.DataArray(np.arange(3), [("x", range(3))])

#--- only operate available elements
print(arr)
print(arr[:-1])
print(arr + arr[:-1])

<xarray.DataArray (x: 3)>
array([0, 1, 2])
Coordinates:
  * x        (x) int64 0 1 2
<xarray.DataArray (x: 2)>
array([0, 1])
Coordinates:
  * x        (x) int64 0 1
<xarray.DataArray (x: 2)>
array([0, 2])
Coordinates:
  * x        (x) int64 0 1
